## Homework

In this homework, we'll deploy the bees vs wasps model we trained in the 
[previous homework](../08-deep-learning/homework.md).

Download the model from here: 

https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

In [1]:
%%capture

!wget https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

## Question 1

Now convert this model from Keras to TF-Lite format.

In [3]:
import tensorflow as tf
from tensorflow import keras

2023-11-19 11:44:58.274249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 11:44:59.157848: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 11:44:59.157901: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 11:44:59.161432: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 11:44:59.471628: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 11:44:59.475649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [9]:
MODEL_PATH: str = 'bees-wasps.h5'

model = keras.models.load_model(MODEL_PATH)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

# Save the model.
with open(f'{MODEL_PATH}.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpl42tuw_b/assets


INFO:tensorflow:Assets written to: /tmp/tmpl42tuw_b/assets
2023-11-19 11:45:59.797527: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-19 11:45:59.797591: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-19 11:45:59.797913: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpl42tuw_b
2023-11-19 11:45:59.799470: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-19 11:45:59.799488: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpl42tuw_b
2023-11-19 11:45:59.802968: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-19 11:45:59.895335: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpl42tuw_b
2023-11-19 11:45:59.908159: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [11]:
!ls -lh *.tflite

-rwxrwxrwx 1 erick erick 43M Nov 19 11:46 bees-wasps.h5.tflite


What's the size of the **converted** model?

* 21 Mb
* **43 Mb**
* 80 Mb
* 164 Mb

## Question 2

To be able to use this model, we need to know the index of the input and the index of the output. 

In [6]:
import tensorflow.lite as tflite

In [13]:
# Load the TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path=f'{MODEL_PATH}.tflite')
interpreter.allocate_tensors()

#input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

What's the output index for this model?

* 3
* 7
* **13**
* 24

## Preparing the image

You'll need some code for downloading and resizing images. You can use this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

In [15]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

Let's download and resize this image: 

https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

Based on the previous homework, what should be the target size for the image?

In [16]:
img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
img = prepare_image(img, (150, 150))




## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

In [24]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
X = np.array(img)
X = X/255
X[0, 0, 0]

0.9450980392156862

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3450980
* 0.5450980
* 0.7450980
* **0.9450980**


## Question 4

Now let's apply this model to this image. What's the output of the model?

In [42]:
X = np.expand_dims(X, 0)

model.predict(X)[0]

1/1 [==============================] - 0s 49ms/step


array([0.65898305], dtype=float32)

* 0.258
* 0.458
* **0.658**
* 0.858


## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

In [44]:
!docker image pull agrigorev/zoomcamp-bees-wasps:v2

[2023-11-19T17:32:48.720490700Z][docker-credential-desktop.exe.system][W] Windows version might not be up-to-date: El sistema no puede encontrar el archivo especificado.
v2: Pulling from agrigorev/zoomcamp-bees-wasps

68a79b8a: Already exists 
124cce46: Already exists 
8b038848: Already exists 
7e7c1be9: Already exists 
8c0b7487: Already exists 
0580071d: Already exists 
881fd503: Pull complete .75MB/40.75MBBDigest: sha256:823f8536a45968f40ee3daf8a2da030b914912a382a4611610b3b84d36d2924c
Status: Downloaded newer image for agrigorev/zoomcamp-bees-wasps:v2
docker.io/agrigorev/zoomcamp-bees-wasps:v2


In [64]:
!docker images agrigorev/zoomcamp-bees-wasps

REPOSITORY                      TAG       IMAGE ID       CREATED        SIZE
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   44 hours ago   662MB


So what's the size of this base image?

* 162 Mb
* 362 Mb
* **662 Mb**
* 962 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

```bash
docker build -t 09-homework .
docker run -it --rm -p 8080:8080 09-homework
```

In [73]:
import requests
import json

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

event = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

response = requests.post(url, json=event).json()

print(json.dumps(response, indent=4))


{
    "prediction": 0.4453350603580475
}


What's the output from the model?

* 0.2453
* **0.4453**
* 0.6453
* 0.8453


## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -t zoomcamp-bees-wasps -f homework.dockerfile .
docker tag zoomcamp-bees-wasps:latest agrigorev/zoomcamp-bees-wasps:v2
docker push agrigorev/zoomcamp-bees-wasps:v2
```


## Submit the results

- Submit your results here: TBA
- If your answer doesn't match options exactly, select the closest one
- You can submit your solution multiple times. In this case, only the last submission will be used


## Deadline

The deadline for submitting is November 27 (Monday), 23:00 CET. After that the form will be closed.